# Graphical analysis of protein interactions in yeast

The aim of this practical is to examine some uses of graphical analysis in a biological setting. The analyses are identical to those demonstrated in the London Tube Graph examples in the introduction to systems biology lecture.

The approaches we saw in the London Tube Map example involved visual examination of the coloured nodes and edges of the graph. For large biological (or other) graphs, this is not feasible. The graph we will be looking at today is shown here.

![Yeast PIN showing node degree](./src/github.com/kortschak/graphprac/yeast-pin.png)

Questions to be answered for the assignment are written in **bold**.

### Programming language

The practical uses the [Go programming language](https://golang.org/). Go is a statically typed, compiled language, but we will be using it through an interactive environment called Jupyter. This tries to make Go behave as an interpretted language, and this may cause some problems at some stages. If you have any problems, please ask the demonstrators for help.

### Setting up the package imports

Go modularises code into packages (much the same way as other languages, though diffferent languages will use different terms for the same concept). Go packages must be imported before they can be used. In a compiled Go program this *must* happen at the beginning of a source code file, though in jupyter, this is not strictly necessary.

The packages that we will use are for printing, `"fmt"`, logging errors, `"log"`, and performing the graph analyses, `"github.com/kortschak/graphprac"`. The first two packages are provided with the language and the last is a small package that wraps graph routines made available through the graph packages of the https://gonum.org project, [`graph/...`](https://godoc.org/gonum.org/v1/gonum/graph/).

In [ ]:
import (
    "fmt"
    "log"

    "github.com/kortschak/graphprac"
)

The documentation for the `"github.com/kortschak/graphprac"` package is available from the godoc.org website: http://godoc.org/github.com/kortschak/graphprac. You can click on links on the documentation page to see the source of the functions and to link to the definitions of types and externally provided functions. This is an excellent way to get to understand the code.

The routines wrapped by the `"github.com/kortschak/graphprac"` are in [`graph/network`](https://godoc.org/gonum.org/v1/gonum/graph/network) and [`graph/community`](https://godoc.org/gonum.org/v1/gonum/graph/community).

### Read in a graph

The graph is from a [data set](http://vlado.fmf.uni-lj.si/pub/networks/data/bio/Yeast/Yeast.htm) used in an analysis of topological structure in the yeast protein interaction network (DOI:[10.1093/nar/gkg340](https://doi.org/10.1093/nar/gkg340)).

In [ ]:
const infile = "./src/github.com/kortschak/graphprac/YeastL.dot"

In [ ]:
g, err := graphprac.NewGraph(infile)
if err != nil {
    log.Fatalf("failed to read graph: %v", err)
}
fmt.Printf("G has %d nodes and %d edges.\n", len(g.Nodes()), len(g.Edges()))

**Notice that the number of nodes agrees with the data set summary linked above, but the number of edges disagrees. Suggest why this might be.**

### Examining the nodes

There is no `head` function in Go (we could write one, but the code is short, so there is no need).

The following code loops over the first 10 elements of the nodes slice (essentially and array - there are differences in Go, but that is not important here) and prints out the node value stored in `n`.

Note that running this multiple times will result in different sets of nodes being printed as the nodes returned by `g.Nodes()` are selected in a random order, so the first ten will differ. Try this out.

**Why is it not important that a node list be returned in a specific order?**

In [ ]:
for _, n := range g.Nodes()[:10] {
    fmt.Println(n)
}

### Network analysis

We are going to look at nodes that have a high connectivity or potential for information flow through the network.

Two measure that we can use to do this (*very roughly*) are node betweenness centrality and PageRank.

The routines provided in the `graphprac` package write their analysis results into the graph that is provided as a parameter. This is not how we normally do this kind of analysis, but it make the practical simpler.

The writing into the graph is done via a set of attributes that have names that can be queried using the functions in the package.

**How can an attribute be queried? Use an example to demonstrate.**

The first analysis is for node betweenness of G.

In [ ]:
graphprac.Betweenness(g)

In [ ]:
nodes, err := graphprac.NodesByAttribute("betweenness", g)
if err != nil {
    log.Fatalf("failed to obtain nodes: %v", err)
}
bestBetween := nodes[0]
for _, n := range nodes[:10] {
    attr := n.Attributes
    fmt.Printf("%s %s -- %s\n", n.Name, attr.Get("betweenness"), attr.Get("desc"))
}

**If you rerun the code block above, does the order/set of nodes change like in the previous example? Why?**

Next we perform a PageRank analysis of G. There are two additional parameters here, `damp` and `tol`.

**What is the purpose of these two parameters? _Hint: You will need to read about PageRank._**

In [ ]:
graphprac.PageRank(g, 0.85, 1e-4)

Now we print out the ten highest ranked nodes and keep the highest ranked node of all.

In [ ]:
nodes, err := graphprac.NodesByAttribute("rank", g)
if err != nil {
    log.Fatalf("failed to obtain nodes: %v", err)
}
bestRank := nodes[0]
for _, n := range nodes[:10] {
    attr := n.Attributes
    fmt.Printf("%s %s -- %s\n", n.Name, attr.Get("rank"), attr.Get("desc"))
}

**Look at the two sets of highest ranked nodes from the betweenness and PageRank analyses. How well do they agree? How does this situation compare to the case of the London Tube Graph example? Why do you think this is?**

The next step is to identify sets of nodes that interact more strongly within the set than they do between sets. These sets are called communities.

![Modular graph showing communities](http://journals.plos.org/ploscompbiol/article/figure/image?size=large&id=10.1371/journal.pcbi.1000190.g004 "Müller-Linow et al PLoS Comp Biol 2008:e1000190")

The `graphprac.Communities` function take a single extra parameter, `resolution`. We are using a resolution of 10.

In [ ]:
graphprac.Communities(g, 10)

Now we are going to identify the community that the highest PageRanked node is in. Note that the algorithm used to identify communities is a randomised algorithm, the [Louvain Algorithm](https://en.wikipedia.org/wiki/Louvain_Modularity), and so different runs will produce a different name for the community and may include slightly different community memberships.

*Community detection is an NP-hard problem and the Louvain Algorithm gives us a reasonable approximation in reasonable time.*

In [ ]:
nodes, err := graphprac.NodesByAttribute("community", g)
if err != nil {
    log.Fatalf("failed to get community: %v", err)
}
comm := ""
for _, n := range nodes {
    attr := n.Attributes
    if n.Name == bestRank.Name {
        comm = attr.Get("community")
        break
    }
}
fmt.Printf("%s is in community %s\n", bestRank.Name, comm)

In [ ]:
for _, n := range nodes {
    attr := n.Attributes
    if attr.Get("community") == comm {
        fmt.Printf("%s -- %s\n", n.Name, attr.Get("desc"))
    }
}

**What are the functions of these proteins? Would you expect them to be in the same community? Hint:http://www.yeastgenome.org/**

**What happens when you alter the resolution parameter to `graphprac.Communities`?**

Finally we are going to look for potential targets to disrupt the function of this community. By looking for edges that have a high edge betweenness we may be able to identify candidates for molecular disruption.

**Run the two cells below and choose an interaction pair that looks like it might be a good candidate for a druggable target. Write a page (~300 words) explaining why the target was chosen and how it could potentially be investigated further. Give references for information you introduce.**

In [ ]:
graphprac.EdgeBetweenness(g)

In [ ]:
edges, err := graphprac.EdgesByAttribute("edge_betweenness", g)
if err != nil {
    log.Fatalf("failed to obtain nodes: %v", err)
}
for _, e := range edges {
    fattr := e.F.Attributes
    tattr := e.T.Attributes
    if fattr.Get("community") != comm && tattr.Get("community") != comm {
        continue
    }
    attr := e.Attributes
    fmt.Printf("%s--%s %s (%s--%s)\n", e.F.Name, e.T.Name, attr.Get("edge_betweenness"), fattr.Get("desc"), tattr.Get("desc"))
}